In [2]:
# !pip install langchain langchain_core langchain_groq langchain_community

In [3]:
# !pip install langgraph

In [4]:
# !pip install requests reportlab googlemaps python-dotenv

In [ ]:
import os
import re
import json
import time
import requests
from urllib.parse import quote
from typing import List, Dict, Any, TypedDict, Annotated
from IPython.display import display, HTML, Markdown
import ipywidgets as widgets
from langchain_core.messages import HumanMessage, AIMessage
from langchain_core.prompts import ChatPromptTemplate
from langchain_groq import ChatGroq
from langgraph.graph import StateGraph, END
from geopy.geocoders import Nominatim
from geopy.extra.rate_limiter import RateLimiter
import folium
from folium.plugins import MarkerCluster
from geopy.geocoders import Nominatim
from geopy.extra.rate_limiter import RateLimiter
import time
from dotenv import load_dotenv



In [ ]:
# ===========================
# CONFIGURATION
# ===========================

load_dotenv()
GROQ_API_KEY = os.getenv("GROQ_API_KEY")

llm = ChatGroq(
    temperature=0.6,
    groq_api_key=GROQ_API_KEY,
    model_name="llama-3.3-70b-versatile"
)

# ===========================
# STATE
# ===========================
class PlannerState(TypedDict):
    messages: Annotated[List[HumanMessage | AIMessage], "Conversation"]
    city: str
    interests: List[str]
    days: int
    itinerary: List[Dict[str, Any]]

# ===========================
# PROMPT
# ===========================
itinerary_prompt = ChatPromptTemplate.from_messages([
    (
        "system",
        """
        You are an expert travel planner and itinerary generator. 
        Generate a detailed, realistic {days}-day travel itinerary for {city}, tailored to the traveler's interests: {interests}. 

        Requirements:
        1. Each day must include multiple activities. Each activity MUST have:
           - 'time': time in 12-hour format (e.g., "09:00 AM")
           - 'place_name': specific landmark, museum, park, restaurant, or attraction
           - 'category': one of ["Sightseeing", "Food", "Shopping", "Culture", "Nature", "Nightlife", "Adventure"]
           - 'description': short, informative description (3-4 sentences) based on {city} an {interests}
        2. Logical sequence: activities should follow a reasonable route (no back-and-forth across city).
        3. Opening hours should be considered for places (no museums at midnight, etc.)
        4. Notes per day can include tips, weather, or travel advice.

        Return output in **valid JSON only** as a list of days:
        [
            {{
                "day": 1,
                "activities": [
                    {{"time": "09:00 AM", "place_name": "Gateway of India", "category": "Sightseeing", "description": "Visit the iconic arch monument by the sea."}},
                    ...
                ],
                "notes": "..."
            }},
            ...
        ]
        DO NOT include any text outside the JSON.
        """
    ),
    ("human", "Generate the itinerary.")
],template_format="f-string")


# ===========================
# CLEAN JSON
# ===========================
def parse_llm_json(text):
    cleaned = re.sub(r"```(?:json)?", "", text)
    cleaned = cleaned.replace("```", "").strip()
    try:
        return json.loads(cleaned)
    except Exception:
        return [{"day": 1, "activities": [{"time": "All day", "place_name": "Unknown", "description": text}], "notes": ""}]

# ===========================
# WEATHER (Free API)
# ===========================
def get_weather(city):
    try:
        url = f"https://wttr.in/{quote(city)}?format=j1"
        data = requests.get(url, timeout=5).json()
        cond = data["current_condition"][0]
        desc = cond["weatherDesc"][0]["value"]
        temp = cond["temp_C"]
        return f"{desc}, {temp}°C"
    except Exception:
        return "Weather data unavailable"

# ===========================
# ACCURATE GOOGLE MAPS LINK
# ===========================
def create_google_maps_link(place_name: str, city: str) -> str:
    """Create precise Google Maps search URL using place name + city."""
    if not place_name:
        return f"https://www.google.com/maps/search/?api=1&query={quote(city)}"
    query = f"{place_name}, {city}"
    return f"https://www.google.com/maps/search/?api=1&query={quote(query)}"

# ===========================
# REVIEW + CORRECT
# ===========================
def review_itinerary(itinerary_json):
    review_prompt = ChatPromptTemplate.from_messages([
        ("system", "You are a travel expert. Review and fix the following itinerary for realism, logical flow, and accurate place naming."),
        ("human", f"Here is the itinerary:\n{json.dumps(itinerary_json, indent=2)}\nEnsure each activity has a valid 'place_name' field.")
    ])
    try:
        response = llm.invoke(review_prompt.format_messages())
        return parse_llm_json(response.content)
    except Exception:
        return itinerary_json




# ===========================
# DISPLAY HTML
# ===========================

def display_itinerary(itinerary, city):
    html = f"""
    <style>
        .itinerary-container {{
            font-family: 'Segoe UI', sans-serif;
            max-width: 850px;
            margin: 30px auto;
            line-height: 1.6;
            color: #1f2937;
        }}
        .day-card {{
            background: #f9fafb;
            border-radius: 15px;
            padding: 20px 25px;
            margin-bottom: 25px;
            box-shadow: 0 3px 10px rgba(0,0,0,0.08);
            transition: transform 0.2s ease;
        }}
        .day-card:hover {{
            transform: translateY(-3px);
        }}
        .day-header {{
            font-size: 1.4em;
            color: #111827;
            margin-bottom: 15px;
            font-weight: 600;
        }}
        .activity {{
            margin: 10px 0;
            padding: 10px 14px;
            border-left: 4px solid #4f46e5;
            background: #ffffff;
            border-radius: 8px;
            box-shadow: 0 1px 3px rgba(0,0,0,0.05);
        }}
        .activity b {{
            color: #111;
        }}
        .category {{
            font-size: 0.9em;
            color: #6b7280;
            margin-left: 5px;
        }}
        .notes {{
            background: #e0e7ff;
            padding: 10px 15px;
            border-radius: 10px;
            margin-top: 15px;
            font-size: 0.95em;
        }}
        a.map-link {{
            text-decoration: none;
            color: #2563eb;
            font-weight: 500;
            margin-left: 5px;
        }}
        a.map-link:hover {{
            text-decoration: underline;
        }}
        h2 {{
            text-align: center;
            color: #111827;
            margin-bottom: 25px;
            font-size: 1.8em;
        }}
        .emoji {{
            font-size: 1.1em;
        }}
    </style>

    <div class="itinerary-container">
        <h2>🌍 Your Smart Trip Itinerary for {city.title()}</h2>
    """

    for day in itinerary:
        html += f"<div class='day-card'>"
        html += f"<div class='day-header'>🗓️ Day {day.get('day', '?')}</div>"

        for act in day.get("activities", []):
            place = act.get("place_name", "Unknown Place")
            time_slot = act.get("time", "")
            desc = act.get("description", "")
            category = act.get("category", "")
            link = f"https://www.google.com/maps/search/?api=1&query={quote(place + ', ' + city)}" if place else "#"

            html += (
                f"<div class='activity'>"
                f"<span class='emoji'>🕒</span> <b>{time_slot}</b> — "
                f"<b>{place}</b> <span class='category'>[{category}]</span>: {desc} "
                f"<a class='map-link' href='{link}' target='_blank'>🔗 View on Maps</a>"
                f"</div>"
            )

        if day.get("notes"):
            html += f"<div class='notes'>📝 {day['notes']}</div>"

        html += "</div>"

    html += "</div>"
    display(HTML(html))

def display_itinerary_map(itinerary, city):
    print("🗺️ Generating interactive maps... (this may take a few seconds)\n")

    geolocator = Nominatim(user_agent="travel_itinerary_mapper_v2")
    geocode = RateLimiter(geolocator.geocode, min_delay_seconds=1, max_retries=2, error_wait_seconds=2.0)

    # Colors for different days
    colors = ["red", "green", "blue", "purple", "orange", "darkred", "cadetblue"]

    # ----------------------------
    # 1. Single Overview Map
    # ----------------------------
    try:
        city_loc = geocode(city, timeout=10)
        fmap_overview = folium.Map(location=[city_loc.latitude, city_loc.longitude], zoom_start=12)
    except:
        fmap_overview = folium.Map(location=[19.0760, 72.8777], zoom_start=12)  # fallback: Mumbai

    overview_cluster = MarkerCluster().add_to(fmap_overview)

    # ----------------------------
    # 2. Separate Map per Day
    # ----------------------------
    for day in itinerary:
        print(f"🗺️ Map for Day {day['day']}")
        try:
            city_loc = geocode(city, timeout=10)
            fmap_day = folium.Map(location=[city_loc.latitude, city_loc.longitude], zoom_start=12)
        except:
            fmap_day = folium.Map(location=[19.0760, 72.8777], zoom_start=12)

        for act in day.get("activities", []):
            place = act.get("place_name", "")
            desc = act.get("description", "")
            if not place:
                continue
            try:
                loc = geocode(f"{place}, {city}", timeout=10)
                if loc:
                    folium.Marker(
                        [loc.latitude, loc.longitude],
                        popup=f"<b>{place}</b><br>{desc}",
                        tooltip=place,
                        icon=folium.Icon(color=colors[(day["day"] - 1) % len(colors)], icon="map-marker")
                    ).add_to(fmap_day)
            except Exception as e:
                print(f"⚠️ Skipping {place}: {e}")
                continue

        display(folium.Html(f"<h3>📍 Day {day['day']} Map</h3>", script=True))
        display(fmap_day)
        
# ===========================
# CREATE ITINERARY PIPELINE
# ===========================
def create_itinerary(state: PlannerState) -> PlannerState:
    print(f"✨ Creating a {state['days']}-day itinerary for {state['city']}...\n")
    
    response = llm.invoke(
        itinerary_prompt.format_messages(
            city=state["city"],
            days=state["days"],
            interests=", ".join(state["interests"])
        )
    )
    itinerary_json = parse_llm_json(response.content)

    # Add weather data
    weather = get_weather(state["city"])
    for day in itinerary_json:
        day["notes"] = (day.get("notes", "") + f" | Weather: {weather}").strip(" |")

    # Review and improve
    itinerary_json = review_itinerary(itinerary_json)

    # Update state
    state["itinerary"] = itinerary_json
    state["messages"].append(AIMessage(content=json.dumps(itinerary_json, indent=2)))

    # Display
    display_itinerary(itinerary_json, state["city"])
    display_itinerary_map(itinerary_json, state["city"])

    return state

# ===========================
# INPUT HANDLER (ipywidgets)
# ===========================
def collect_inputs():
    city_box = widgets.Text(placeholder="Enter city (e.g., Tokyo, Mumbai, Paris)")
    days_box = widgets.BoundedIntText(value=3, min=1, max=7, description="Days:")
    interests_box = widgets.Text(placeholder="e.g., food, culture, beaches")
    button = widgets.Button(description="🧭 Generate Itinerary", button_style="success")

    display(Markdown("### ✈️ AI Travel Planner"))
    display(city_box, days_box, interests_box, button)

    state = {
        "messages": [],
        "city": "",
        "interests": [],
        "days": 1,
        "itinerary": []
    }

    def on_click(b):
        city = city_box.value.strip()
        if not city:
            print("❌ Please enter a valid city.")
            return
        state["city"] = city
        state["days"] = days_box.value
        state["interests"] = [i.strip() for i in interests_box.value.split(",") if i.strip()]
        state["messages"].append(HumanMessage(content=f"Plan trip to {state['city']} for {state['days']} days with interests {state['interests']}"))
        create_itinerary(state)

    button.on_click(on_click)

# ===========================
# RUN
# ===========================
collect_inputs()


### ✈️ AI Travel Planner

Text(value='', placeholder='Enter city (e.g., Tokyo, Mumbai, Paris)')

BoundedIntText(value=3, description='Days:', max=7, min=1)

Text(value='', placeholder='e.g., food, culture, beaches')

Button(button_style='success', description='🧭 Generate Itinerary', style=ButtonStyle())

✨ Creating a 4-day itinerary for Mumbai...



🗺️ Generating interactive maps... (this may take a few seconds)

🗺️ Map for Day 1


🗺️ Map for Day 2


🗺️ Map for Day 3


🗺️ Map for Day 4
